In [3]:
from bs4 import BeautifulSoup
import os

from aeneas.executetask import ExecuteTask
from aeneas.task import Task

root = "workspace/SaveTheCat_epub/"

# Folder containing the XHTML files
infolder_path = root + "plaintext"
outfolder_path = root + "sync_text"

# Initialize ID counter
id_counter = 1

# Iterate through each file in the folder
for filename in sorted(os.listdir(infolder_path), key = lambda x: x[:4]):
    if filename.endswith('.xhtml'):
        infile_path = os.path.join(infolder_path, filename)
        outfile_path = os.path.join(outfolder_path, filename)

        # Read the XHTML file
        with open(infile_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Parse the XHTML content using BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')

        # Unwrap all <span class="RunIn"> elements
        for spanx in soup.find_all('span', class_='RunIn'):
            spanx.unwrap()

        # Save the modified content back to the file
        with open(outfile_path, 'w', encoding='utf-8') as file:
                file.write(str(soup))
                
# Folder containing the XHTML files
infolder_path = root + "sync_text"
outfolder_path = root + "sync_text"

# Initialize ID counter
id_counter = 1

# Iterate through each file in the folder
for filename in sorted(os.listdir(infolder_path), key = lambda x: x[:4]):
    if filename.endswith('.xhtml'):
        infile_path = os.path.join(infolder_path, filename)
        outfile_path = os.path.join(outfolder_path, filename)

        # Read the XHTML file
        with open(infile_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Parse the XHTML content using BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')

        # Unwrap all <span class="RunIn"> elements
        for spanx in soup.find_all('span', class_='RunIn'):
            spanx.unwrap()

        # Find the <body> element
        body_element = soup.find('body')

        # Find all text elements within the <body> element
        text_elements = body_element.find_all(string=True)

        # Iterate through the text elements and wrap them with span tags and unique IDs
        for element in text_elements:
    
            if '.' in element:  # Check if there's a dot in the text

                parts = element.split('.')
                new_parts = []
                for i, part in enumerate(parts):
                    if part.strip():
                        span_tag = soup.new_tag('span')
                        span_tag['id'] = f'f{str(id_counter).zfill(6)}'
                        span_tag.string = part.strip() 

                        if i < len(parts) -1:
                            if parts[i+1].strip():
                                if parts[i+1][0] == "”":
                                    span_tag.string += '.” '
                                    parts[i+1] = parts[i+1][1:]
                                else:
                                    span_tag.string += '. '
                            else:
                                    span_tag.string += '. '

                        else: 
                            if span_tag.string[-1] != "“":
                                span_tag.string += ' '

                        new_parts.append(span_tag)
                        id_counter += 1
                    else:
                        new_parts.append(part)
                element.replace_with(*new_parts)
            else:
                if element.strip():
                    span_tag = soup.new_tag('span')
                    span_tag['id'] = f'f{str(id_counter).zfill(6)}'
                    element.wrap(span_tag)
                    id_counter += 1

        # Save the modified content back to the file
        with open(outfile_path, 'w', encoding='utf-8') as file:
            file.write(str(soup))
            

# audiofiles = sorted(os.listdir(root + "audio"), key = lambda x: x[:2])
# synctextfiles = sorted(os.listdir(root + "sync_text"), key = lambda x: x[:4])

audiofiles = sorted(os.listdir(root + "audio"), key = lambda x: x[:2])[:1]

synctextfiles = sorted(os.listdir(root + "sync_text"), key = lambda x: x[:4])[:1]

assert len(audiofiles) == len(synctextfiles)

print(*zip(audiofiles, synctextfiles ))

for i in range(len(audiofiles)):
    print(i)
    # create Task object
    config_string = u"task_language=eng|os_task_file_format=smil|os_task_file_smil_audio_ref=../audio/{0}|os_task_file_smil_page_ref=../text/{1}|is_text_type=unparsed|is_text_unparsed_id_regex=f[0-9]+|is_text_unparsed_id_sort=numeric".format(audiofiles[i], synctextfiles[i])
    task = Task(config_string=config_string)
    task.audio_file_path_absolute =  root + u"audio/" + audiofiles[i]
    task.text_file_path_absolute = root + u"sync_text/" + synctextfiles[i]
    task.sync_map_file_path_absolute = root + "/smil/{}.smil".format(synctextfiles[i])

    # process Task
    Execute = ExecuteTask(task)
    Execute.logger.tee = True
    Execute.execute()

    # output sync map to file
    task.output_sync_map_file()

('02. 002.mp3', 'c00.xhtml')
0
Create Task
Executeing task
[DEBUG] 2024-05-09 13:37:54.141155 ExecuteTask: Executing task...
[DEBUG] 2024-05-09 13:37:54.141464 ExecuteTask: Both audio and text input file are present
[DEBUG] 2024-05-09 13:37:54.141490 ExecuteTask: Executing single level task...
[DEBUG] 2024-05-09 13:37:54.141503 ExecuteTask: STEP 1 BEGIN (extract MFCC real wave)
[DEBUG] 2024-05-09 13:37:54.141537 AudioFileMFCC: Initializing MFCCs...
[DEBUG] 2024-05-09 13:37:54.141576 AudioFile: Loading audio data...
[DEBUG] 2024-05-09 13:37:54.141919 AudioFile: self.file_format is None or not good => converting self.file_path
[DEBUG] 2024-05-09 13:37:54.142759 AudioFile: Temporary PCM16 mono WAVE file: '/tmp/tmpwcqinrr6.wav'
[DEBUG] 2024-05-09 13:37:54.142792 AudioFile: Converting audio file to mono...
[DEBUG] 2024-05-09 13:37:54.142889 FFMPEGWrapper: Calling with arguments '['ffmpeg', '-i', 'workspace/SaveTheCat_epub/audio/02. 002.mp3', '-ac', '1', '-ar', '16000', '-y', '-map_metadata'

In [19]:
import aeneas
import numpy as np

m1 = np.loadtxt('m1.txt')
m2 = np.loadtxt('m2.txt')
delta = 3000
# print(m1)

result_0 = aeneas.cdtw.cdtw.compute_best_path(m1, m2, delta)
result_0

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 3),
 (2, 3),
 (3, 3),
 (4, 3),
 (5, 3),
 (6, 3),
 (7, 3),
 (8, 3),
 (9, 3),
 (10, 3),
 (11, 3),
 (12, 3),
 (13, 3),
 (14, 3),
 (15, 3),
 (16, 3),
 (17, 3),
 (18, 3),
 (19, 3),
 (20, 3),
 (21, 3),
 (22, 4),
 (23, 4),
 (24, 5),
 (25, 6),
 (25, 7),
 (26, 8),
 (27, 9),
 (28, 10),
 (29, 11),
 (30, 12),
 (31, 13),
 (31, 14),
 (32, 15),
 (33, 16),
 (34, 17),
 (35, 18),
 (35, 19),
 (36, 20),
 (37, 20),
 (37, 21),
 (37, 22),
 (37, 23),
 (37, 24),
 (37, 25),
 (37, 26),
 (38, 26),
 (39, 26),
 (40, 26),
 (41, 26),
 (42, 26),
 (43, 26),
 (44, 26),
 (45, 26),
 (46, 26),
 (47, 26),
 (48, 26),
 (49, 26),
 (50, 26),
 (51, 26),
 (52, 26),
 (53, 26),
 (54, 26),
 (55, 26),
 (56, 26),
 (57, 26),
 (58, 26),
 (59, 26),
 (60, 26),
 (61, 26),
 (62, 26),
 (63, 26),
 (64, 26),
 (65, 26),
 (66, 26),
 (67, 26),
 (68, 26),
 (69, 26),
 (70, 26),
 (71, 26),
 (72, 26),
 (73, 26),
 (74, 26),
 (75, 26),
 (76, 26),
 (77, 26),
 (78, 26),
 (79, 26),
 (80, 26),
 (81, 26),
 (82, 26),


In [18]:
import aeneas_rust
import numpy as np

m1 = np.loadtxt('m1.txt')
m2 = np.loadtxt('m2.txt')
delta = 3000

# result = rust_extension.compute_best_path([[0.1, 0.2], [0.3, 0.4]], [[0.1, 0.2], [0.3, 0.4]], 1)
result = aeneas_rust.compute_best_path(m1, m2, delta)
result

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 3),
 (2, 3),
 (3, 3),
 (4, 3),
 (5, 3),
 (6, 3),
 (7, 3),
 (8, 3),
 (9, 3),
 (10, 3),
 (11, 3),
 (12, 3),
 (13, 3),
 (14, 3),
 (15, 3),
 (16, 3),
 (17, 3),
 (18, 3),
 (19, 3),
 (20, 3),
 (21, 3),
 (22, 4),
 (23, 4),
 (24, 5),
 (25, 6),
 (25, 7),
 (26, 8),
 (27, 9),
 (28, 10),
 (29, 11),
 (30, 12),
 (31, 13),
 (31, 14),
 (32, 15),
 (33, 16),
 (34, 17),
 (35, 18),
 (35, 19),
 (36, 20),
 (37, 20),
 (37, 21),
 (37, 22),
 (37, 23),
 (37, 24),
 (37, 25),
 (37, 26),
 (38, 26),
 (39, 26),
 (40, 26),
 (41, 26),
 (42, 26),
 (43, 26),
 (44, 26),
 (45, 26),
 (46, 26),
 (47, 26),
 (48, 26),
 (49, 26),
 (50, 26),
 (51, 26),
 (52, 26),
 (53, 26),
 (54, 26),
 (55, 26),
 (56, 26),
 (57, 26),
 (58, 26),
 (59, 26),
 (60, 26),
 (61, 26),
 (62, 26),
 (63, 26),
 (64, 26),
 (65, 26),
 (66, 26),
 (67, 26),
 (68, 26),
 (69, 26),
 (70, 26),
 (71, 26),
 (72, 26),
 (73, 26),
 (74, 26),
 (75, 26),
 (76, 26),
 (77, 26),
 (78, 26),
 (79, 26),
 (80, 26),
 (81, 26),
 (82, 26),


In [16]:
result[:30000] == result_0[:30000]

True

In [15]:
len(result), len(result_0)

(33781, 32207)

In [17]:
result[32207:]

[(27707, 20138),
 (27707, 20139),
 (27707, 20140),
 (27707, 20141),
 (27707, 20142),
 (27707, 20143),
 (27707, 20144),
 (27707, 20145),
 (27707, 20146),
 (27707, 20147),
 (27707, 20148),
 (27707, 20149),
 (27707, 20150),
 (27707, 20151),
 (27707, 20152),
 (27707, 20153),
 (27707, 20154),
 (27707, 20155),
 (27707, 20156),
 (27707, 20157),
 (27707, 20158),
 (27707, 20159),
 (27707, 20160),
 (27707, 20161),
 (27707, 20162),
 (27707, 20163),
 (27707, 20164),
 (27707, 20165),
 (27707, 20166),
 (27707, 20167),
 (27707, 20168),
 (27707, 20169),
 (27707, 20170),
 (27707, 20171),
 (27707, 20172),
 (27707, 20173),
 (27707, 20174),
 (27707, 20175),
 (27707, 20176),
 (27707, 20177),
 (27707, 20178),
 (27707, 20179),
 (27707, 20180),
 (27707, 20181),
 (27707, 20182),
 (27707, 20183),
 (27707, 20184),
 (27707, 20185),
 (27707, 20186),
 (27707, 20187),
 (27707, 20188),
 (27707, 20189),
 (27707, 20190),
 (27707, 20191),
 (27707, 20192),
 (27707, 20193),
 (27707, 20194),
 (27707, 20195),
 (27707, 20196

In [ ]:
import IPython.display as ipd
import numpy as np
m1 = np.loadtxt('m1.txt')
m2 = np.loadtxt('m2.txt')
# Load a local WAV file
# ipd.Audio("SaveTheCat_epub/audio/" + audiofiles[0])
ipd.Audio(m1, rate=16000)